In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Read the training data 
data = pd.read_csv('../input/widsdatathon2022/train.csv')

### Initial look at the data

**Initial thoughts on the data**
* The data is at year and building level 
* There are around 75k records in the training data
* There is temperature information for each month as separate predictors. The energy usage can depend on the weather which can be determined by the months. So, having this information separately is good.
* There's also information on the number of days the temperature was on the extreme ends (warm or cold)

**Initial theories** 
* Older homes might have higher EUIs as most of the appliances might not be power efficient
* Winter and summer months might see higher EUIs due to usage of heaters and A/Cs
* Apartments with more floor area might see higher EUIs as more power will be required to heat up or cool down the apartment. 

In [3]:
data.head()

In [4]:
data.describe()

In [5]:
data.columns

In [6]:
data.info()

In [7]:
numerical_predictors = data.select_dtypes(include = [np.number]).columns
categorical_predictors = data.select_dtypes(exclude = [np.number]).columns

#### Data distribution of individual explanatory variables

Numerical variables

In [8]:
plt.figure(figsize = (20, 90))
for idx, col in enumerate(numerical_predictors):
    plt.subplot(32, 2, idx + 1)
    sns.histplot(data[col], kde = True)
    plt.title(f'Distribution of {col}')
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
plt.show()
    

In [9]:
# year_built
bins = [1900, 1920, 1940, 1960, 1980, 2000, 2021]
groups = pd.cut(data['year_built'], bins)
groups.value_counts().plot(kind = 'bar')
groups.value_counts()
# Distribution of year_built across the years

In [10]:
# Floor area
data.boxplot(column = 'floor_area', figsize = (10,7))

In [11]:
# Elevation
data.boxplot(column = 'ELEVATION', figsize = (10,7))

**Observations**
* The data distribution is skewed for most of the predictors
* Over 30% of the buildings in the data are the oldest - are built in the range 1920-1940
* The floor area is not really variable (steep peak for the mean) - most of the bigger values are outliers that makes sense. Same for elevation.
* The temperature columns don't have a specific data distribution. The data seems really random with a lot of gaps in between.
* The wind-related columns seem to have a huge gap in the middle before having some data that might be considered as outliers.


Categorical variables

In [12]:
plt.figure(figsize = (20, 20))
for idx, col in enumerate(categorical_predictors):
    plt.subplot(2, 2, idx + 1)
    data[col].value_counts().plot(kind = 'bar')
    plt.title(f'Distribution of {col}')
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

**Observations**
* The dataset seems to have more data from State 6. Will this cause a bias? Probably shouldn't use this as a predictor. But, does EUI depend on government laws in some way?
* Multifamily type seems to be the facility type with most records in the dataset.

#### Relation between site_eui and explanatory variables

Categorical variables

In [13]:
data.boxplot(by = 'building_class', column = 'site_eui', figsize = (10,7), showfliers = False)

In [14]:
data.boxplot(by = 'State_Factor', column = 'site_eui', figsize = (10,7))

In [15]:
data.boxplot(by = 'facility_type', column = 'site_eui', figsize = (20,10), rot = 90)

**Observations**
* The building class does see a difference in median for residential and commercial properties - but not by much. For categorical variables we want classes for which variance is high for response as that will help in differentiating between the values of response. The building class seems to be okay (not great) - can still include in the model.
* The state data was biased in terms of the number of records in the dataset. Probably not a good idea to include this.
* There is a lot of variability in the different classes of facility type but the data distribution for this was also highly biased. So, not sure how much this can be trusted. However, can create custom classes and divide the existing ones depending on the number of records or type of facility.

Numerical variables

In [16]:
plt.rcParams['figure.figsize'] = [20,100]
plt.rcParams['figure.dpi'] = 100
fig, axs = plt.subplots(32,2)

for itr, col in zip(axs.flat, numerical_predictors): # Flatten the axes or use ax[row,col] format in the loop
    sns.regplot(x='site_eui', y=col, data=data, line_kws={"color":"green"}, ax=itr)
fig.tight_layout()
plt.show()  

#### Correlation between explanatory variables

In [17]:
# Find correlation 
figsize=(100,150)
df_num = data[numerical_predictors]
corr = df_num.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(figsize))
sns.heatmap(corr, mask = mask, cmap = 'vlag')

In [18]:
non_temp_predictors = ['Year_Factor', 'floor_area', 'year_built', 'energy_star_rating',
       'ELEVATION', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inches',
       'snowdepth_inches', 'avg_temp', 'days_below_30F', 'days_below_20F',
       'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F',
       'days_above_100F', 'days_above_110F', 'direction_max_wind_speed',
       'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog',
       'site_eui']

figsize=(30,30)
df_num = data[non_temp_predictors]
corr = df_num.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(figsize))
sns.heatmap(corr, mask = mask, cmap = 'coolwarm')

**Observations** 
The below columns seem to be highly correlated - 
* snowdepth_inches and snowfall_inches
* direction_peak_wind_speed and direction_max_wind_speed
* max_wind_speed and direction_max_wind_speed

#### Look for missing values

In [19]:
# Look for missing data 
data.columns[data.isnull().any()]

In [20]:
data[['year_built', 'energy_star_rating', 'direction_max_wind_speed',
       'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog']].isnull().sum()

The columns - 'direction_max_wind_speed', 'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog' - have missing values for around 60% of the total data. So, ignoring these as predictors might be better than imputation or filtering the missing values. 

For the other 2 fields, year_built and energy_star_rating-
* For energy_start_rating, there's still around 30% of data missing so imputation will be a better option than filtering out rows for which it's value is NaN. The decision to remove this column is something that needs to be considered later as from the description it feels like something that's worth a try. 
* Filtering out rows with year_built with missing value might work in this case or not using this field. The field will not be used directly thought; something like age of the building can be derived to be used as predictor.

In [21]:
plt.rcParams['figure.figsize'] = [20,10]
data['energy_star_rating'].plot.hist(bins=range(1,100))

In [22]:
data['year_built'].plot.hist(xlim = [1900,2022], bins = 500)

In [23]:
data[['year_built', 'energy_star_rating']].describe()

In [24]:
# Data transformation to handle missing values
data = data[data['year_built'].notnull()].copy()
data['energy_star_rating'].fillna(data['energy_star_rating'].mean(), inplace = True)

In [25]:
from datetime import date
data['building_age'] = data.apply(lambda x: date.today().year - x['year_built'], axis = 1)

### Model Selection

Planning to use all the columns for now except - facility_type (needs more analysis),year_built,'direction_max_wind_speed', 'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog'

In [26]:
# Transform categorical variables
data = pd.get_dummies(data, columns = ['building_class'], drop_first = True)

In [27]:
# Data Transformation before fitting model
all_predictors = set(data.columns)
remove_predictors = ['id','Year_Factor','State_Factor', 'facility_type','year_built','direction_max_wind_speed','direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog', 'site_eui']
predictors = list(all_predictors - set(remove_predictors))

In [28]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score

X = data[predictors]
y = data['site_eui']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [29]:
def evaluate_model(y_test, y_pred):
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f'RMSE : {RMSE}')
    print(f'R2 : {r2}')
    n = X_test.shape[0]
    p = X_test.shape[1]
    adj_r2 = 1 - ((1-r2)*(n-1)/(n-p-1))
    print(f'Adjusted-R2 : {adj_r2}')

### Multilinear regression

The basic model that can be tried is **Multi-Linear Regression** to fit a line through each predictor and response plot an get coefficients that indicates the change in response with unit change in each predictor

In [30]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [31]:
for name, coef in zip(predictors, lr_model.coef_):
    print(f'{name}: {coef}')

In [32]:
print(f'Intercept : {lr_model.intercept_}')

In [33]:
pred_values_lr = lr_model.predict(X_test)
evaluate_model(y_test,pred_values_lr )

### Random Forest

In [34]:
#Random Forest 

from sklearn.ensemble import RandomForestRegressor
# Values after using grid search
rf_model = RandomForestRegressor(n_estimators=500, min_samples_split = 2, min_samples_leaf = 2, max_depth = 15)
rf_model.fit(X_train, y_train)

pred_values_rf = rf_model.predict(X_test)
evaluate_model(y_test,pred_values_rf )

### Gradient Boost

In [35]:
# Gradient boost 
from sklearn.ensemble import GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=500)
gb_model.fit(X_train, y_train)

pred_values_gb = gb_model.predict(X_test)
evaluate_model(y_test, pred_values_gb)

##### Grid search code

In [ ]:
# Grid search for Random Forest 
#forest = RandomForestRegressor(random_state = 1)
#from sklearn.model_selection import GridSearchCV
#n_estimators = [100, 300, 500]
#max_depth = [5, 8, 15]
#min_samples_split = [2, 5, 10]
#min_samples_leaf = [2, 5, 10] 

#hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
#              min_samples_split = min_samples_split, 
#             min_samples_leaf = min_samples_leaf)

#gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
#                      n_jobs = -1, scoring = 'neg_root_mean_squared_error')
#bestF = gridF.fit(X_train, y_train)

In [ ]:
#bestF.best_params_

In [ ]:
#from sklearn.metrics import SCORERS
#sorted(SCORERS.keys())

In [ ]:
# Grid search for Gradient Boost
forest = GradientBoostingRegressor(random_state = 1)
from sklearn.model_selection import GridSearchCV
n_estimators = [100, 300, 500]
max_depth = [5, 8, 15]
min_samples_split = [2, 5, 10]
min_samples_leaf = [2, 5, 10] 
learning_rate = [0.1, 0.01, 1, 0.5]

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf, learning_rate = learning_rate)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1, scoring = 'neg_root_mean_squared_error')
bestF = gridF.fit(X_train, y_train)

In [ ]:
bestF.best_params_